In [ ]:
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 26kB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 3.9MB 45.5MB/s 
     |████████████████████████████████| 450kB 38.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=6b93ac7e274ef5cd1c17528b4615d9aeba0419987b0e74d8ead566fb103167c5
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
from numpy import argmax
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
x=np.zeros((294,256,256,3),dtype=np.float32)
y=np.zeros((294,256,256,1),dtype=np.float32)

In [ ]:
p=0
for im in os.listdir('/content/drive/MyDrive/segd'):
    if im.endswith('.jpg'):
        for seg in os.listdir('/content/drive/MyDrive/segd'):
            if im[14:-4] == seg[14:-8] and seg.endswith('.png'):
                img=cv2.imread('/content/drive/MyDrive/segd'+'/'+im)
                sege=cv2.imread('/content/drive/MyDrive/segd'+'/'+seg,0)
                img = cv2.resize(img,(256,256))
                sege = cv2.resize(sege,(256,256))
                sege=np.reshape(sege,(256,256,1))
                x[p]=img
                y[p]=sege
                p=p+1

p

294

In [ ]:
np.save('/content/drive/MyDrive/proj/x_sky.npy', x)
np.save('/content/drive/MyDrive/proj/y_sky.npy', y)

In [ ]:
x=np.load('/content/drive/MyDrive/proj/x_sky.npy')
y=np.load('/content/drive/MyDrive/proj/y_sky.npy')

In [ ]:
x=x.astype('float32')
y=y.astype('float32')
y=y/255.0

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [ ]:

IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3


#Build the model
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])



In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,batch_size=64,epochs=500)

Train on 264 samples
Epoch 1/500
264/264 [==============================] - 3s 11ms/sample - loss: 0.0252 - accuracy: 0.9876
Epoch 2/500
264/264 [==============================] - 1s 5ms/sample - loss: 0.0250 - accuracy: 0.9877
Epoch 3/500
264/264 [==============================] - 1s 5ms/sample - loss: 0.0251 - accuracy: 0.9877
Epoch 4/500
264/264 [==============================] - 1s 5ms/sample - loss: 0.0250 - accuracy: 0.9877
Epoch 5/500
264/264 [==============================] - 1s 5ms/sample - loss: 0.0251 - accuracy: 0.9877
Epoch 6/500
264/264 [==============================] - 1s 5ms/sample - loss: 0.0250 - accuracy: 0.9877
Epoch 7/500
264/264 [==============================] - 1s 5ms/sample - loss: 0.0252 - accuracy: 0.9876
Epoch 8/500
264/264 [==============================] - 1s 5ms/sample - loss: 0.0250 - accuracy: 0.9876
Epoch 9/500
264/264 [==============================] - 1s 5ms/sample - loss: 0.0249 - accuracy: 0.9877
Epoch 10/500
264/264 [=============================

KeyboardInterrupt: ignored

In [ ]:
model.save